In [0]:
from pyspark.sql import functions as F
from delta.tables import DeltaTable
from pyspark.sql.window import Window

In [0]:
%run /Workspace/Ecommerce/consolidate_pipeline/Setup/utilities

In [0]:
dbutils.widgets.text("catalog","fmcg","Catalog")
dbutils.widgets.text("data_source","gross_price","Data source")

In [0]:
catalog = dbutils.widgets.get("catalog")
data_source = dbutils.widgets.get("data_source")
base_path = f's3://child-sportsbar-cd/{data_source}/*csv'
print(base_path)

In [0]:
df = (
    spark.read.format('csv')
         .option('header',True)
         .option('inferSchema',True)
         .load(base_path)
         .withColumn('read_timestamp',F.current_timestamp())
         .select("*","_metadata.file_name","_metadata.file_size")
)

display(df.limit(10))

In [0]:
df.printSchema()

### Bronze layer processing

In [0]:
df.write\
    .format('delta')\
    .option("delta.enableChangeDataFeed","true")\
    .mode('overwrite')\
    .saveAsTable(f"{catalog}.{bronze_schema}.{data_source}")

### Silver layer Processing

In [0]:
df_bronze = spark.sql(f"select * from {catalog}.{bronze_schema}.{data_source}")


#### Transfoormations

In [0]:
df_bronze.select('month').distinct().show()

In [0]:
date_formats = ["yyyy/MM/dd", "dd/MM/yyyy", "yyyy-MM-dd", "dd-MM-yyyy"]

df_silver = df_bronze.withColumn(
    'month',
    F.coalesce(
        F.try_to_date(F.col("month"),"yyyy/MM/dd"),
        F.try_to_date(F.col("month"),"dd/MM/yyyy"),
        F.try_to_date(F.col("month"),"yyyy-MM-dd"),
        F.try_to_date(F.col("month"),"dd-MM-yyyy")
    )
)

In [0]:
df_silver.select('month').distinct().show()

In [0]:
df_silver = df_silver.withColumn(
    "gross_price",
    F.when(F.col("gross_price").rlike(r'^-?\d+(\.\d+)?$'), 
           F.when(F.col("gross_price").cast("double") < 0, -1 * F.col("gross_price").cast("double"))
            .otherwise(F.col("gross_price").cast("double")))
    .otherwise(0)
)

In [0]:
df_silver.show(10)

In [0]:
# fetching data from fmcg.silver.proucts

df_products = spark.table("fmcg.silver.products")

df_joined = df_silver.join(df_products.select("product_id","product_code"), on="product_id",how="inner")
df_joined = df_joined.select("product_id","product_code","month","gross_price","read_timestamp","file_name","file_size")
df_joined.show(10)

In [0]:
df_joined.write\
 .format("delta") \
 .option("delta.enableChangeDataFeed", "true")\
 .option("mergeSchema", "true") \
 .mode("overwrite") \
 .saveAsTable(f"{catalog}.{silver_schema}.{data_source}")
 

#### Gold layer processing

In [0]:
df_silver = spark.sql(f"SELECT * FROM {catalog}.{silver_schema}.{data_source}")

In [0]:
df_gold = df_silver.select("product_code","month","gross_price")
df_gold.show(5)


In [0]:
df_gold.write\
 .format("delta") \
 .option("delta.enableChangeDataFeed", "true") \
 .mode("overwrite") \
 .saveAsTable(f"{catalog}.{gold_schema}.sb_dim_{data_source}")

In [0]:
df_gold_price = spark.table("fmcg.gold.sb_dim_gross_price")

In [0]:
df_gold_price = (
    df_gold_price
    .withColumn("year", F.year("month"))
    .withColumn("is_zero", F.when(F.col("gross_price") == 0, 1).otherwise(0))
)

w = (
    Window
    .partitionBy("product_code", "year")
    .orderBy(F.col("is_zero"), F.col("month").desc())
)


df_gold_latest_price = (
    df_gold_price
      .withColumn("rnk", F.row_number().over(w))
      .filter(F.col("rnk") == 1)
)

In [0]:
df_gold_latest_price = df_gold_latest_price.select("product_code", "year", "gross_price").withColumnRenamed("gross_price", "price_inr").select("product_code", "price_inr", "year")

df_gold_latest_price = df_gold_latest_price.withColumn("year", F.col("year").cast("string"))

df_gold_latest_price.show(5)

In [0]:
delta_table = DeltaTable.forName(spark, "fmcg.gold.dim_gross_price")


delta_table.alias("target").merge(
    source=df_gold_latest_price.alias("source"),
    condition="target.product_code = source.product_code"
).whenMatchedUpdate(
    set={
        "price_inr": "source.price_inr",
        "year": "source.year"
    }
).whenNotMatchedInsert(
    values={
        "product_code": "source.product_code",
        "price_inr": "source.price_inr",
        "year": "source.year"
    }
).execute()